**Measuring Demand Forecasting benefits series**

# Traditional forecasting baseline

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio, and the default `ml.t3.medium` instance type (2 vCPU + 4 GiB RAM)*

To estimate the benefits of a new demand forecasting model, we need a pre-existing baseline forecast to compare against. Many businesses without access to ML expertise use moving average-based estimates for forecasting their demand, so this notebook prepares a basic forecast using that approach.

We'll retrospectively calculate the "forecasts" back from the start of the historical dataset, through to one period after the historical data finishes.

First, import necessary libraries and load the raw generated sales data:

In [ ]:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:
import os

# External Dependencies:
import numpy as np
import pandas as pd


# Load raw sales data:
df = pd.read_parquet('s3://measuring-forecast-benefits-assets/dataset/v1/sales.parquet')

CAT_DIMENSIONS = [name for name in df.columns if pd.api.types.is_object_dtype(df[name].dtype)]
START_DATE = df['date'].min()
END_DATE = df['date'].max()

print(f'Categorical fields: {CAT_DIMENSIONS}')
df.head()

## Calculating the moving average forecast

Let's start by aggregating our values per month, as a daily forecast is usually not feasible in a traditional forecasting context. This step can take a few extra seconds to complete:

In [ ]:
%%time

df['month'] = df['date'].dt.strftime('%Y-%m')
END_MONTH = END_DATE.strftime('%Y-%m')

df_monthly = df.drop(columns=['date']).groupby(
    ['month'] + CAT_DIMENSIONS
).agg({'sales': 'sum'}).reset_index()

df_monthly

Note that (like most sales data) the source does not include a record on days where a particular item/location/etc combination had zero sales. For infrequently-selling items, there may also be whole months without sales.

- For our rolling average to be accurate and avoid over-forecasting, we should ensure all *historical* combinations are represented, with zeros for any missing data
- To build a useful **forward-looking** forecast, we'd like to also extend the dataframe into the future, but leave `sales` as unknown (`NaN`) in those future period(s)

In [ ]:
FORECAST_MONTHS = 1  # Number of months to forecast

# List of all months from start to end of global dataset:
all_months_index = pd.date_range(
    START_DATE,
    END_DATE + pd.DateOffset(months=FORECAST_MONTHS),
    freq='M',
    inclusive='both',
)

# Cartesian product of all months with all present categorical field combinations:
all_combos = pd.merge(
    all_months_index.to_series(name='month').dt.strftime('%Y-%m'),
    df_monthly[CAT_DIMENSIONS].drop_duplicates(),
    how='cross'
)

# Join the monthly DF with the full combination list, and fill any *historical* NAs with 0:
df_monthly = pd.merge(df_monthly, all_combos, on=['month'] + CAT_DIMENSIONS, how='outer')
df_monthly.loc[df_monthly['sales'].isnull() & (df_monthly['month'] <= END_MONTH), 'sales'] = 0
df_monthly

Double-check that only forward-looking months have NaN sales:

In [ ]:
print('Months with unknown sales:')
df_monthly[df_monthly['sales'].isna()]["month"].drop_duplicates()

We're now ready to calculate the rolling average. Note that:

- The `ROLLING_MONTHS` parameter lets you define the number of past months you want to consider...
- ...But this introduces a **cold start problem**: Because a rolling average forecast is not defined for the initial months where you have no historical data.
- We need to exclude the *current* month from the average, because it's supposed to be a forecast!

In [ ]:
ROLLING_MONTHS = 6

# Must make sure the records are sorted by month before rolling:
df_monthly = df_monthly.sort_values('month').reset_index(drop=True)

df_monthly['mov_avg'] = df_monthly.groupby(CAT_DIMENSIONS)['sales'].transform(
    lambda group: np.round(
        group.rolling(
            window=ROLLING_MONTHS,
            closed='left',  # Exclude current month from average (as it's a forecast)
            min_periods=int(ROLLING_MONTHS/2),
        ).mean()
    )
)
df_monthly

As mentioned above, you'll see that the forecast is undefined for the first few months until enough historical data is available:

In [ ]:
print('Months with NaN mov_avg:')
df_monthly[df_monthly['mov_avg'].isna()]['month'].unique()

...But at the end of the dataset in the forward-looking month(s), the moving average is available even though the `sales` is `NaN`.

This flat forecast file is the format we need for dashboarding and analysis in later notebooks, so we'll export it to CSV:

In [ ]:
os.makedirs('dataset', exist_ok=True)
df_monthly.dropna().drop(columns=['sales']).to_csv('dataset/moving_avg.csv', index=False)

For easier reading, we could also view this data in a more traditional pivoted view. First the historical sales:

> ⚠️ **Note:** We filter on historical months here because the default Pandas view will summarize `NaN` in future months to `0.0`, which is misleading

In [ ]:
historical_sales = df_monthly[df_monthly['month'] <= END_MONTH].pivot_table(
    index=CAT_DIMENSIONS, columns='month', values='sales', aggfunc=np.sum
)
historical_sales

...and second the moving average forecast for each month:

Again you'll see `0.0` below for the first months before the moving average is available here, which is a little misleading - these entries are `NaN` in the underlying data.

In [ ]:
movavg_pivot = df_monthly.pivot_table(
    index=CAT_DIMENSIONS, columns='month', values='mov_avg', aggfunc=np.sum
)
movavg_pivot

## Next steps

In this notebook we prepared a basic moving-average demand forecast from historical sales data, to give a baseline for comparing business benefits of different forecasting models.

Next, you can follow along to **either**:

- [1.2. Run SageMaker Canvas.ipynb](1.2.%20Run%20SageMaker%20Canvas.ipynb) - to build an ML-based demand forecast through a business analyst-accessible UI with Amazon SageMaker Canvas
- Or [1.3. Run Amazon Forecast.ipynb](1.3.%20Run%20Amazon%20Forecast.ipynb) - to build an ML-based demand forecast through the AWS Python SDK and Amazon Forecast

The results should be similar, so there's no need to run both - you can choose whichever tool is the best fit for your team's skill-set. 

> ⚠️ **But before you go:** If you're using the default `ml.t3.medium` (2 vCPU + 4 GiB RAM) instance type, you may want to **stop or restart this notebook's kernel** to free up memory before you move on. If you're not sure how to do this, just click the "Restart the kernel" circular arrow icon (⤾) up in the toolbar.